# Part 2: Training

In this part we will show how using MLRun's **Feature Store** we can easily define a **Feature Vector** and create the dataset we need to run our training process.  
By the end of this tutorial you’ll learn how to:
- Combine multiple data sources to a single Feature Vector
- Create training dataset
- Create a model using an MLRun Hub function

In [1]:
import mlrun

project_name, _ = mlrun.set_environment(project='fraud-demo', 
                                        user_project=True)

> 2021-08-26 03:30:00,149 [info] loaded project fraud-demo-admin from MLRun DB


## Create Feature Vector  
In this section we will create our Feature Vector.  
The Feature vector will have a `name` so we can reference to it later via the URI or our serving function, and a list of `features` from the available FeatureSets.  We can add a feature from a feature set by adding `<FeatureSet>.<Feature>` to the list, or add `<FeatureSet>.*` to add all the FeatureSet's available features.  

By default, the first FeatureSet in the feature list will act as the spine. meaning that all the other features will be joined to it.  
For example, in this instance we use the early_sense sensor data as our spine, so for each early_sense event we will create produce a row in the resulted Feature Vector.

In [2]:
# Define the list of features we will be using
features = ['transactions.amount_max_2h', 
            'transactions.amount_sum_2h', 
            'transactions.amount_count_2h',
            'transactions.amount_avg_2h', 
            'transactions.amount_max_12h', 
            'transactions.amount_sum_12h',
            'transactions.amount_count_12h', 
            'transactions.amount_avg_12h', 
            'transactions.amount_max_24h',
            'transactions.amount_sum_24h', 
            'transactions.amount_count_24h', 
            'transactions.amount_avg_24h',
            'transactions.es_transportation_count_14d', 
            'transactions.es_health_count_14d',
            'transactions.es_otherservices_count_14d', 
            'transactions.es_food_count_14d',
            'transactions.es_hotelservices_count_14d', 
            'transactions.es_barsandrestaurants_count_14d',
            'transactions.es_tech_count_14d', 
            'transactions.es_sportsandtoys_count_14d',
            'transactions.es_wellnessandbeauty_count_14d', 
            'transactions.es_hyper_count_14d',
            'transactions.es_fashion_count_14d', 
            'transactions.es_home_count_14d', 
            'transactions.es_travel_count_14d', 
            'transactions.es_leisure_count_14d',
#             'transactions.age_mapped',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_day_of_week',
            'events.*']

In [3]:
# Import MLRun's Feature Store
import mlrun.feature_store as fs

# Define the feature vector name for future reference
feature_vector_name = 'transactions-fraud'

# Define the feature vector using our Feature Store (fs)
fv = fs.FeatureVector(feature_vector_name, 
                      features, 
                      label_feature="labels.label",
                      description='Predicting a fraudulent transaction')

# Save the feature vector in the Feature Store
fv.save()

## Produce training dataset 

In [4]:
# Import the Parquet Target so we can directly save our dataset as a file
from mlrun.datastore.targets import ParquetTarget

# Get offline feature vector
# will return a pandas dataframe and save the dataset to parquet so a 
# training job can train on it
dataset = fs.get_offline_features(feature_vector_name, target=ParquetTarget())

> 2021-08-26 03:30:01,047 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-admin/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'status': 'ready', 'updated': '2021-08-26T03:30:01.047546+00:00', 'size': 1947835}


In [5]:
# Preview our dataset
df = dataset.to_dataframe()

In [6]:
df.tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,amount_sum_12h,amount_count_12h,amount_avg_12h,amount_max_24h,amount_sum_24h,...,gender_F,gender_M,step,amount,timestamp_hour,timestamp_day_of_week,event_details_change,event_login,event_password_change,label
49996,37.40,37.40,1.0,37.40,37.40,51.90,2.0,25.950000,45.84,172.25,...,0,1,40,37.40,3,3,0.0,0.0,1.0,0
49997,14.78,14.78,1.0,14.78,14.78,14.78,1.0,14.780000,14.78,14.78,...,1,0,17,14.78,3,3,0.0,0.0,1.0,0
49998,19.69,19.69,1.0,19.69,50.08,103.23,4.0,25.807500,50.08,130.56,...,0,1,115,19.69,3,3,0.0,0.0,1.0,0
49999,60.41,60.41,1.0,60.41,84.98,332.87,8.0,41.608750,238.69,604.83,...,1,0,122,60.41,3,3,0.0,0.0,1.0,0
50000,26.95,26.95,1.0,26.95,26.95,69.08,3.0,23.026667,26.95,88.22,...,0,1,59,26.95,3,3,0.0,1.0,0.0,0


## Run training with AutoML as a cluster job

In [7]:
from mlrun.platforms import auto_mount

# Import the SKLearn based training function from our functions hub
fn = mlrun.import_function('hub://sklearn-classifier').apply(auto_mount())

In [8]:
# Prepare the parameters list for the training function
# We use 3 different models to test on our dataset
model_list = {"model_name": ['transaction_fraud_rf', 
                             'transaction_fraud_xgboost', 
                             'transaction_fraud_adaboost'],
              
              "model_pkg_class": ['sklearn.ensemble.RandomForestClassifier',
                                  'sklearn.ensemble.GradientBoostingClassifier',
                                  'sklearn.ensemble.AdaBoostClassifier']}

# Define the training task, including our feature vector, label and hyperparams definitions
task = mlrun.new_task('training', 
                      inputs={'dataset': fv.uri},
                      params={'label_column': 'label'}
                     )

task.with_hyper_params(model_list, strategy='list', selector='max.accuracy')

# Run the function 
fn.spec.image = 'mlrun/mlrun'
train = fn.run(task, local=False)

> 2021-08-26 03:30:01,560 [info] starting run training uid=f87698dfa3cc4f7697c883f03f33039e DB=http://mlrun-api:8080
> 2021-08-26 03:30:01,736 [info] Job is running in the background, pod: training-kfbgw
> 2021-08-26 03:30:20,349 [info] best iteration=1, used criteria max.accuracy
> 2021-08-26 03:30:20,826 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...3f33039e,0,Aug 26 03:30:06,completed,training,v3io_user=adminkind=jobowner=admin,dataset,label_column=label,best_iteration=1accuracy=0.9930769230769231test-error=0.006923076923076923rocauc=0.944160942328456brier_score=0.005725365384615385f1-score=0.7534246575342465precision_score=0.8870967741935484recall_score=0.6547619047619048,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-08-26 03:30:20,935 [info] run executed, status=completed


## Perform feature selection process on a sample of the dataset (Using mlrun marketplace function)

As part of our data science process we will try and reduce the training dataset's size to get rid of bad or unuseful features and save computation time.

We will use our ready-made feature selection function from our hub [`hub://feature_selection`](https://github.com/mlrun/functions/blob/development/feature_selection/feature_selection.ipynb) to select the best features to keep on a sample from our dataset and run the function on that.


In [9]:
feature_selection_fn = mlrun.import_function('hub://feature_selection')
feature_selection_fn.apply(auto_mount())

feature_selection_run = feature_selection_fn.run(
            params={'k': 5, 'sample_ratio':0.25,
                    'output_vector_name':feature_vector_name + "-short",
                   'ignore_type_errors': True},
            inputs={'df_artifact': fv.uri},
            name='feature_extraction',
            handler='feature_selection',
    local=False)

> 2021-08-26 03:30:21,161 [info] starting run feature_extraction uid=bbc101ecfaf7426c90c894c19e634f74 DB=http://mlrun-api:8080
> 2021-08-26 03:30:21,313 [info] Job is running in the background, pod: feature-extraction-vtps9
> 2021-08-26 03:30:29,550 [info] Couldn't calculate chi2 because of: Input X must be non-negative.
> 2021-08-26 03:30:33,514 [info] votes needed to be selected: 3
> 2021-08-26 03:30:34,163 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-admin/FeatureStore/transactions-fraud-short/parquet/vectors/transactions-fraud-short-latest.parquet', 'status': 'ready', 'updated': '2021-08-26T03:30:34.163882+00:00', 'size': 669979}
> 2021-08-26 03:30:34,262 [info] run executed, status=completed
Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
Liblinear failed to converge, increase the number of iterations.
Liblinear failed to converge, increase the number of iterations.
final 

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...9e634f74,0,Aug 26 03:30:26,completed,feature_extraction,v3io_user=adminkind=jobowner=adminhost=feature-extraction-vtps9,df_artifact,k=5sample_ratio=0.25output_vector_name=transactions-fraud-shortignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-admin/transactions-fraud-short,f_classifmutual_info_classiff_regressionLinearSVCLogisticRegressionExtraTreesClassifierfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


> 2021-08-26 03:30:36,539 [info] run executed, status=completed


In [10]:
top_features_df = mlrun.get_dataitem(feature_selection_run.outputs['top_features_vector']).as_df()
top_features_df.tail()

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,label
49996,37.40,37.40,1.0,37.40,37.40,0
49997,14.78,14.78,1.0,14.78,14.78,0
49998,19.69,19.69,1.0,19.69,50.08,0
49999,60.41,60.41,1.0,60.41,84.98,0
50000,26.95,26.95,1.0,26.95,26.95,0


## Train ensemble of models with top features

In [11]:
# Defining our training task, including our feature vector, label and hyperparams definitions
task = mlrun.new_task('training', 
                      inputs={'dataset': feature_selection_run.outputs['top_features_vector']},
                      params={'label_column': 'label'}
                     )
task.with_hyper_params(model_list, strategy='list', selector='max.accuracy')

run = fn.run(task)

> 2021-08-26 03:30:36,595 [info] starting run training uid=3fa156c559ed450f8f21f7954dd03aa5 DB=http://mlrun-api:8080
> 2021-08-26 03:30:36,746 [info] Job is running in the background, pod: training-xtbwx
> 2021-08-26 03:30:49,145 [info] best iteration=3, used criteria max.accuracy
> 2021-08-26 03:30:49,610 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-admin,...4dd03aa5,0,Aug 26 03:30:41,completed,training,v3io_user=adminkind=jobowner=admin,dataset,label_column=label,best_iteration=3accuracy=0.992214204329662test-error=0.007785795670338018rocauc=0.9591472232934823brier_score=0.2039113382689779f1-score=0.7092198581560285precision_score=0.819672131147541recall_score=0.625,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-08-26 03:30:55,936 [info] run executed, status=completed
